### Tutorial showing how to read EBAS NASA Ames files

**Note**: this notebook is currently under development

Please see [here](https://ebas-submit.nilu.no/Submit-Data/Getting-started) for information related to the EBAS NASA Ames file format.

**Further links**:
 - [Pyaerocom website](http://aerocom.met.no/pyaerocom/)
 - [Pyaerocom installation instructions](http://aerocom.met.no/pyaerocom/readme.html#installation)
 - [Getting started](http://aerocom.met.no/pyaerocom/notebooks.html#getting-started)

In [1]:
import os 
from pyaerocom.io import EbasNasaAmesFile

ebasdir = "/lustre/storeA/project/aerocom/aerocom1/AEROCOM_OBSDATA/EBASMultiColumn/data/data/"
filename = "DE0043G.20080101000000.20160708144500.nephelometer..aerosol.1y.1h.DE09L_tsi_neph_3563.DE09L_nephelometer.lev2.nas"

mc = EbasNasaAmesFile(file=ebasdir+filename,
                      only_head=False,          #set True if you only want to import header
                      replace_invalid_nan=True, #replace invalid values with NaNs
                      convert_timestamps=True,  #compute datetime64 timestamps from numerical values
                      decode_flags=True,        #decode all flags (e.g. 0.111222333 -> 111 222 333)
                      verbose=False)
print(mc)

2018-09-21 11:36:46,200:WARNING:
basemap extension library is not installed (or cannot be imported. Some features will not be available


Elapsed time init all variables: 0.026292085647583008 s


2018-09-21 11:36:46,944:WARNING:
geopy library is not available. Aeolus data read not enabled


2018-09-21 11:36:47,100:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


Elapsed time init pyaerocom: 1.0720207691192627 s


Pyaerocom EbasNasaAmesFile
--------------------------

   num_head_lines: 60
   num_head_fmt: 1001
   data_originator: Flentje, Harald
   sponsor_organisation: DE09L, Deutscher Wetterdienst, DWD, Met. Obs., Hohenspeissenberg, , 82283, Hohenspeissenberg, Germany
   submitter: Flentje, Harald
   project_association: EUSAAR GAW-WDCA
   vol_num: 1
   vol_totnum: 1
   ref_date: 2008 01 01 2016 07 08
   revision_date: nan
   freq: 0.041667
   descr_time_unit: days from file reference point
   num_cols_dependent: 11
   mul_factors (list, 11 items)
   [1.0
    1.0
    ...
    1.0
    1.0]

   vals_invalid (list, 11 items)
   [999.999999
    999.999
    ...
    9999.9
    9.999999999]

   descr_first_col: end_time of measurement, days from the file reference point

   Column variable definitions
   -------------------------------
   EbasColDef: name=starttime, unit=days, is_var=False, is_flag=False, flag_id=, 
   EbasColDef: name=endtime, unit=days, is_var=False, is_flag=False, flag_id=, 
   Eb

#### Column information

In [2]:
mc.print_col_info()

Column 0
Pyaerocom EbasColDef
--------------------
name: starttime
unit: days
is_var: False
is_flag: False
flag_id: 

Column 1
Pyaerocom EbasColDef
--------------------
name: endtime
unit: days
is_var: False
is_flag: False
flag_id: 

Column 2
Pyaerocom EbasColDef
--------------------
name: aerosol_light_backscattering_coefficient
unit: 1/Mm
is_var: True
is_flag: False
flag_id: numflag
wavelength: 450.0 nm

Column 3
Pyaerocom EbasColDef
--------------------
name: aerosol_light_backscattering_coefficient
unit: 1/Mm
is_var: True
is_flag: False
flag_id: numflag
wavelength: 550.0 nm

Column 4
Pyaerocom EbasColDef
--------------------
name: aerosol_light_backscattering_coefficient
unit: 1/Mm
is_var: True
is_flag: False
flag_id: numflag
wavelength: 700.0 nm

Column 5
Pyaerocom EbasColDef
--------------------
name: aerosol_light_scattering_coefficient
unit: 1/Mm
is_var: True
is_flag: False
flag_id: numflag
wavelength: 450.0 nm

Column 6
Pyaerocom EbasColDef
--------------------
name: aerosol_l

You can see that all variable columns were assigned the same flag column, since there is only one. This would be different if there were multiple flag columns (e.g. one for each variable).

#### Access flag information

You can access the flags for each column using the ``flags`` attribute of the file.

In [3]:
mc.flags

OrderedDict([('numflag',
              <pyaerocom.io.ebas_nasa_ames.EbasFlagCol at 0x7f36700d9908>)])

In [4]:
flagcol = mc.flags["numflag"]

The raw flags can be accessed via:

In [5]:
flagcol.raw_data

array([0.394999, 0.394999, 0.394999, ..., 0.247   , 0.247   , 0.      ])

And the processed flags are in stored in a (Nx3) numpy array where N is the total number of timestamps.

In [6]:
flagcol.flags

array([[394, 999,   0],
       [394, 999,   0],
       [394, 999,   0],
       ...,
       [247,   0,   0],
       [247,   0,   0],
       [  0,   0,   0]])

For instance, access the flags of the 5 timestamp:

In [7]:
flagcol.flags[4]

array([394, 999,   0])

#### Convert object to pandas Dataframe

The conversion does currently exclude all flag columns

In [8]:
df = mc.to_dataframe()
df

,aerosol_light_backscattering_coefficient,aerosol_light_backscattering_coefficient,aerosol_light_backscattering_coefficient,aerosol_light_scattering_coefficient,aerosol_light_scattering_coefficient,aerosol_light_scattering_coefficient,pressure,relative_humidity,temperature
2008-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 01:29:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 02:29:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 03:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 04:29:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 05:29:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 06:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 07:29:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 08:29:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Performance

Read only header

In [9]:
%%timeit
EbasNasaAmesFile(file=ebasdir+filename,
                 only_head=True, verbose=False)

2018-09-21 11:36:47,324:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:47,330:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:47,335:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:47,340:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:47,345:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


...

2018-09-21 11:36:52,853:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:52,860:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:52,867:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:52,874:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


6.81 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Read raw:

In [10]:
%%timeit
EbasNasaAmesFile(file=ebasdir+filename,
                      only_head=False,          #set True if you only want to import header
                      replace_invalid_nan=False, #replace invalid values with NaNs
                      convert_timestamps=False,  #compute datetime64 timestamps from numerical values
                      decode_flags=False,        #decode all flags (e.g. 0.111222333 -> 111 222 333)
                      verbose=False)

2018-09-21 11:36:53,244:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:53,311:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:53,369:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:53,428:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:53,485:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


...

2018-09-21 11:36:57,597:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:57,644:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:57,703:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:57,750:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


56.3 ms ± 9.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Perform all operations:

In [11]:
%%timeit
EbasNasaAmesFile(file=ebasdir+filename,
                      only_head=False,          #set True if you only want to import header
                      replace_invalid_nan=True, #replace invalid values with NaNs
                      convert_timestamps=True,  #compute datetime64 timestamps from numerical values
                      decode_flags=True,        #decode all flags (e.g. 0.111222333 -> 111 222 333)
                      verbose=False)

2018-09-21 11:36:57,851:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:57,940:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:58,019:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:58,087:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:36:58,145:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


...

2018-09-21 11:37:02,765:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:37:02,835:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:37:02,890:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


2018-09-21 11:37:02,944:WARNING:
Failed to read header row 6.
2008 01 01 2016 07 08

Error msg: IndexError('list index out of range',)


63 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
